In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as sig

Target: 6
Probe: 3

In [76]:
ds1 = pd.read_csv("P3TE/loading_data/data_faces_target_6_10t_29i_l3.csv")
labs1 = np.loadtxt("P3TE/loading_data/labels_faces_target_6_10t_29i_l3.csv",
                   delimiter=',',
                   dtype='int')

In [80]:
ds2 = pd.read_csv("P3Te/loading_data/data_faces_target_6_30t_10i_l3.csv")
labs2 = np.loadtxt("P3TE/loading_data/labels_faces_target_6_30t_10i_l3.csv",
                   delimiter=',', dtype='int')

In [82]:
ds3 = pd.read_csv("P3TE/loading_data/data_faces_target_6_30t_10i_no_click_l3.csv")
labs3 = np.loadtxt("P3TE/loading_data/labels_faces_target_6_30t_10i_no_click_l3.csv",
                   delimiter=',', dtype='int')

In [84]:
rm0102 = ['channel1',
          'channel2',
          'channel3',
          'channel4',
          'channel5',
          'channel6']

In [100]:
bpfilt = sig.butter(4, (1, 12.5), 'bandpass', output='sos', fs=250)

In [155]:
def create_lab_data(ds, labs, channels):
    # get trigger col
    ds_trigger = ds['Trigger'].values

    # get image events
    img_starts = []
    for i in range(len(ds_trigger) - 1):
        if (ds_trigger[i] == 0) and (ds_trigger[i+1] == 1):
            img_starts.append(i+1)
    img_starts = np.array(img_starts)
    
    filt_sigs = []
    for i in channels:
        filt_sigs.append(sig.sosfilt(bpfilt, ds[i]))
    filt_sigs = np.array(filt_sigs)

    # get EEG values averaged across channels (without O1 and O2)
    channel_avg_eeg = []
    for i in img_starts:
        img_obs = filt_sigs[:,i:i+250].mean(axis=0)
        channel_avg_eeg.append(img_obs)
    channel_avg_eeg = np.array(channel_avg_eeg)

    # create labeled data
    eeg_img_ds = pd.DataFrame(channel_avg_eeg, columns=list(range(250)))
    eeg_img_ds['image'] = labs

    return eeg_img_ds

In [160]:
X1 = create_lab_data(ds1, labs1, ['channel4'])
X2 = create_lab_data(ds2, labs2, ['channel4'])
X3 = create_lab_data(ds3, labs3, ['channel4'])

In [346]:
X = pd.concat([X1, X2, X3], ignore_index=True, axis=0)

## Bootstrapping

### Probe, Test, Irrelevant (PTI) Paradigm

In [269]:
test_targets = [0, 1, 2, 4, 5, 6, 7, 8]

for j in test_targets:
    X_target = X.iloc[np.where(X['image'] == 3)]
    X_probe = X.iloc[np.where(X['image'] == j)]

    tester = np.arange(len(X))
    tps = np.concatenate((np.where(X['image'] == j), np.where(X['image'] == 3))).flatten()
    xirr_inds = np.delete(tester, tps)
    X_irr = X.iloc[xirr_inds]

    bootstrap_index = 0
    for i in range(100):
        target_inds = np.random.choice(np.arange(len(X_target)), 50, replace=False)
        probe_inds = np.random.choice(np.arange(len(X_probe)), 50, replace=False)
        irr_inds = np.random.choice(np.arange(len(X_irr)), 50, replace=False)

        xtd = X_target.loc[:,X_target.columns != 'image']
        xpd = X_probe.loc[:,X_probe.columns != 'image']
        xid = X_irr.loc[:,X_irr.columns != 'image']

        Xtsamp = xtd.iloc[target_inds]
        Xpsamp = xpd.iloc[probe_inds]
        Xisamp = xid.iloc[irr_inds]

        Xtot = pd.concat([Xtsamp, Xpsamp, Xisamp], ignore_index=True, axis=0)
        xtotbar = Xtot.mean(axis=0)

        xtargetbar_dc = Xtsamp.mean(axis=0) - xtotbar
        xprobebar_dc = Xpsamp.mean(axis=0) - xtotbar
        xirrbar_dc = Xisamp.mean(axis=0) - xtotbar

        dc_corr_PT = np.corrcoef(xprobebar_dc, xtargetbar_dc)[0,1]
        dc_corr_PI = np.corrcoef(xprobebar_dc, xirrbar_dc)[0,1]

        bootstrap_index += (dc_corr_PT > dc_corr_PI)

    print(j, bootstrap_index)

0 8
1 9
2 1
4 2
5 4
6 46
7 10
8 0


### One-versus all BCD (no target)

In [347]:
X_no_img = X.loc[:,X.columns != 'image']

In [424]:
test_imgs = [0, 1, 2, 3, 4, 5, 7, 8]
bootstrap_results = []

for ti in test_imgs:
    X_test = X_no_img.iloc[np.where(X['image'] == ti)]
    X_others = X_no_img.iloc[np.where(X['image'] != ti)]

    bootstrap_vals = []

    for i in range(100):
        test_samp_ids = np.random.choice(np.arange(len(X_test)), 50, replace=False)
        other_samps_ids = np.random.choice(np.arange(len(X_others)), 50, replace=False)

        X_test_samp = X_test.iloc[test_samp_ids].mean(axis=0)
        X_other_samp = X_others.iloc[other_samps_ids].mean(axis=0)

        dc_corr = np.corrcoef(X_test_samp, X_other_samp)[0,1]
        bootstrap_vals.append(dc_corr)

    bootstrap_results.append(bootstrap_vals)

bootstrap_results = np.array(bootstrap_results)

In [425]:
bootstrap_results.mean(axis=1)

array([0.45746836, 0.44693047, 0.23817556, 0.25345613, 0.42987614,
       0.20955645, 0.41382129, 0.49854638])

## Load new data

In [ ]:
ds1 = pd.read_csv("P3TE/loading_data/data_probe_4_15t_15i_5s_davis.csv")
labs1 = np.loadtxt("P3TE/loading_data/labels_probe_4_15t_15i_5s_davis.csv",
                   delimter=',',
                   dtype='int')